In [3]:
import time
import bosdyn.client 
from bosdyn.client import create_standard_sdk
from bosdyn.client.robot_state import RobotStateClient
from bosdyn.client.robot_id import RobotIdClient 

In [4]:
#Robot Kinematic State with tranform snapshop & timestamp 
class KRobotState:
    def __init__(self,robot): 
        self._robot=robot
        self._robot_state_client=robot.ensure_client(RobotStateClient.default_service_name) 
        self.RobotState=self._robot_state_client.get_robot_state()
        self._transformSnapshot=None
        self._RobotTime=None 
    def getTransformSnapshot(self): 
        self._transformSnapshot= self.RobotState.kinematic_state().transforms_snapshot
        return  self._transformSnapshot
    def GetTimeStamp(self): 
        self._RobotTime=self.RobotState.kinematic_state().acquisition_timestamp
        return self._RobotTime 

In [21]:
from bosdyn.client.frame_helpers import get_a_tform_b 
from bosdyn.client.frame_helpers import VISION_FRAME_NAME,ODOM_FRAME_NAME

In [1]:
#Location data with methods for tranforming from SPOT's Odom and Vision frames
class Location():
    fromOdom=[]
    fromVision=[]
    Time=[]
    def _init_(self,robot,ToFrame="body"):
        self.RobotState=KRobotState(robot)
        self.FromFrame=None 
        self.ToFrame=ToFrame
        self.TransformSnapshot=None
        self.fiducial_objects=None
    def OdomTransform(self): 
        if  self.fiducial_objects!= None:
            self.FromFrame=ODOM_FRAME_NAME
        else:
            self.FromFrame="odom"
            self.TransformSnapshot=self.RobotState.getTransformSnapshot()
            odom=get_a_tform_b(self.TransformSnapshot,self.FromFrame,self.ToFrame)
            self.fromOdom.append(odom) 
    def VisionTransform(self): 
        if  self.fiducial_objects!= None:
            self.FromFrame=VISION_FRAME_NAME
        else: 
            self.FromFrame="vision"
            vision=get_a_tform_b(self.TransformSnapshot,self.FromFrame,self.ToFrame)
            self.fromVision.append(vision)
    def TimeStamp(self): 
        self.Time.append(self.RobotState.GetTimeStamp())
        
            

In [10]:
import datetime

In [4]:
#robot location data 
class RobotLocation(Location):
    def __init__(self,robot):
       super()._init_(robot) 
    def start(self): 
      self.OdomTransform()
      self.VisionTransform()
      self.TimeStamp()
         

In [15]:
from bosdyn.client.world_object import WorldObjectClient 
from bosdyn.api import world_object_pb2

In [2]:
#april tag location data 
class AprilTag(Location): 
    def __init__(self,robot):
        self.world_object_client=robot.ensure_client(WorldObjectClient.default_service_name)
        self.fiducial_objects=None
        self.ToFrame= None 
        super().__init__(robot)
    #request Tag
    def RequestFiducial(self): 
        request_fiducials=[world_object_pb2.WORLD_OBJECT_APRILTAG]
        self.fiducial_objects=self.world_object_client.list_world_objects(object_type=request_fiducials).world_objects
        return self.fiducial_objects
    def AprilOdomTransform(self): 
        if  self.fiducial_objects != None:
            super().OdomTransform()
            for fiducial in self.fiducial_objects:
                self.ToFrame=fiducial.apriltag_properties.frame_name_fiducial
                self.TransformSnapshot= fiducial.transforms_snapshot
                odom=get_a_tform_b(self.TransformSnapshot,super().FromFrame,self.ToFrame) 
                self.fromOdom.append(odom) 
    def AprilVisionTransform(self): 
        if  self.fiducial_objects != None:
            super().VisionTransform()
            for fiducial in self.fiducial_objects:
                self.ToFrame=fiducial.apriltag_properties.frame_name_fiducial
                self.TransformSnapshot= fiducial.transforms_snapshot
                vision=get_a_tform_b(self.TransformSnapshot,super().FromFrame,self.ToFrame)
                self.fromOdom.append(vision)    
    def TagTime(self):
        self.Time.append(datetime.datetime.now()) 
    def start(self): 
        self.RequestFiducial()
        self.AprilOdomTransform()
        self.AprilVisionTransform()
        self.TagTime() 


    

In [2]:
import bosdyn.client.util

In [3]:
import argparse
import pickle 
def main(argv):
    parser=argparse.ArgumentParser()
    bosdyn.client.util.add_common_arguments(parser)
    parser.add_argument('--vs',action='store_true',default=False)
    options=parser.parse_args(argv)
#create robot object to invoke SPOT    
    sdk=create_standard_sdk('test')
    robot=sdk.create_robot(options.host_name)
    robot.authenticate(options.username, options.password)
#time sync 
    robot.time_sync.wait_for_sync() 
#create object to obtain Location Data
    robot_localization= RobotLocation(robot)
    fiducial_localization= AprilTag(robot)
#log robot info 
    
    try: 
        while True: 
            input('Press Enter to log robot and fiducial positions...')
            robot_localization.start()
            fiducial_localization.start()
            with open('RobotData.pickle','wb') as Robotfile: 
                pickle.dump(robot_localization, Robotfile)
            with open('TagData.pickle','wb') as Tagfile: 
                pickle.dump(fiducial_localization, Tagfile)

    except KeyboardInterrupt: 
        print("Caught Keyboard Interupt, exitting")
        return True 

    








    
    

In [11]:
class dummy:
    Alist=[]
    def __init__(self,id,tag=None):
        self.id=id 
        self.var1=id+1
        self.tag=tag
    def meth(self):
        if self.tag != None:
            print(" tag detected") 
        return self.var1

class dummy2(dummy): 
    def __init__(self,id): 
        super().__init__(id)
        self.tag=None 
    def requestTag(self): 
        listTag=[1,2,3,4]
        self.tag=len(listTag)
        return self.tag 
    def meth2(self): 
        a= super().meth()
        self.Alist.append(self.tag)
        print(self.Alist)
#TEST     
obj=dummy2(15)
#obj.requestTag()
obj.meth2()

[None]
